In [10]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs
# from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
# from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultAccessor, run_diba_batch, BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# active_global_batch_result_filename='global_batch_result_2023-07-12.pkl'
active_global_batch_result_filename='global_batch_result_2023-07-07.pkl'

debug_print=True
enable_neptune = False

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
    set_environment_variables(neptune_kwargs)

In [12]:
""" 
from pyphoplacecellanalysis.General.Batch.runBatch import main, BatchRun, run_diba_batch, run_specific_batch

"""
global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

## NEPTUNE:
if enable_neptune:
	project = neptune.init_project(**neptune_kwargs)
	project["general/global_batch_result_filename"] = active_global_batch_result_filename
	project["general/global_data_root_parent_path"] = global_data_root_parent_path.as_posix()

## TODO: load the batch result initially:

## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

if enable_neptune:
	run = neptune.init_run() # rember to run.stop()
	run['parameters/global_batch_result_file_path'] = global_batch_result_file_path.as_posix()
	# project["general/data_analysis"].upload("data_analysis.ipynb")
	run["dataset/latest"].track_files(f"file://{global_batch_result_file_path}") # "s3://datasets/images"

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)

if enable_neptune:
	run["dataset/global_batch_run_progress_df"].upload(File.as_html(batch_progress_df)) # "path/to/test_preds.csv"
	run["dataset/global_batch_run_good_only_df"].upload(File.as_html(good_only_batch_progress_df)) # "path/to/test_preds.csv"
	
batch_progress_df

finalized_loaded_global_batch_result_pickle_path: /media/MAX/Data/global_batch_result_2023-07-07.pkl
Loading loaded session pickle file results : /media/MAX/Data/global_batch_result_2023-07-07.pkl... encountered exception 'BatchRun' object is not iterable while printing. Turning into a warning and continuing.
done.
switching data dir path from /home/halechr/turbo/Data to /media/MAX/Data


,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/kdiba/gor01/one/2006-6-07_11-2...,SessionBatchProgress.COMPLETED,None,45,322,2,65,False,,,
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/kdiba/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,198,40,166,True,,,
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/kdiba/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,42,122,40,81,True,,,
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/kdiba/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.FAILED,need at least one array to concatenate,0,0,0,0,False,,,
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/kdiba/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,30,34,34,True,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,/media/MAX/Data/kdiba/pin01/one/fet11-04_21-20-3,SessionBatchProgress.COMPLETED,None,70,0,66,4,False,,,
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/media/MAX/Data/kdiba/pin01/one/redundant,SessionBatchProgress.FAILED,Required File: /nfs/turbo/umms-kdiba/Data/KDIB...,0,0,0,0,False,,,
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/media/MAX/Data/kdiba/pin01/one/showclus,SessionBatchProgress.FAILED,Required File: /nfs/turbo/umms-kdiba/Data/KDIB...,0,0,0,0,False,,,
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/media/MAX/Data/kdiba/pin01/one/sleep,SessionBatchProgress.FAILED,Required File: /nfs/turbo/umms-kdiba/Data/KDIB...,0,0,0,0,False,,,


In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    # display(batch_progress_df)
    good_only_batch_progress_df.batch_results.build_all_columns()
    display(good_only_batch_progress_df)

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/kdiba/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,198,40,166,True,,,
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/kdiba/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,42,122,40,81,True,,,
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/kdiba/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,30,34,34,True,,,
5,kdiba,gor01,one,2006-6-13_14-42-6,kdiba_gor01_one_2006-6-13_14-42-6,/media/MAX/Data/kdiba/gor01/one/2006-6-13_14-42-6,SessionBatchProgress.COMPLETED,None,43,13,39,23,True,,,
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/kdiba/gor01/two/2006-6-07_16-4...,SessionBatchProgress.COMPLETED,None,42,145,40,239,True,,,
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/media/MAX/Data/kdiba/gor01/two/2006-6-08_21-1...,SessionBatchProgress.COMPLETED,None,40,30,40,47,True,,,
9,kdiba,gor01,two,2006-6-09_22-24-40,kdiba_gor01_two_2006-6-09_22-24-40,/media/MAX/Data/kdiba/gor01/two/2006-6-09_22-2...,SessionBatchProgress.COMPLETED,None,51,78,43,299,True,,,
10,kdiba,gor01,two,2006-6-12_16-53-46,kdiba_gor01_two_2006-6-12_16-53-46,/media/MAX/Data/kdiba/gor01/two/2006-6-12_16-5...,SessionBatchProgress.COMPLETED,None,41,41,40,22,True,,,
11,kdiba,gor01,two,2006-6-13_15-22-3,kdiba_gor01_two_2006-6-13_15-22-3,/media/MAX/Data/kdiba/gor01/two/2006-6-13_15-22-3,SessionBatchProgress.COMPLETED,None,41,20,42,14,True,,,
12,kdiba,vvp01,one,2006-4-09_17-29-30,kdiba_vvp01_one_2006-4-09_17-29-30,/media/MAX/Data/kdiba/vvp01/one/2006-4-09_17-2...,SessionBatchProgress.COMPLETED,None,63,27,44,35,True,,,


In [ ]:
good_only_batch_progress_df.

In [14]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import UserAnnotationsManager

user_is_replay_good_annotations = UserAnnotationsManager.get_user_annotations()

good_only_batch_progress_df['has_user_replay_annotations'] = [(a_row_context.adding_context_if_missing(display_fn_name='DecodedEpochSlices',epochs='replays',decoder='long_results_obj',user_annotation='selections') in user_is_replay_good_annotations) for a_row_context in good_only_batch_progress_df['context']]
good_only_batch_progress_df

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/kdiba/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,40,198,40,166,True,,,,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/kdiba/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,42,122,40,81,True,,,,False
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/kdiba/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,40,30,34,34,True,,,,True
5,kdiba,gor01,one,2006-6-13_14-42-6,kdiba_gor01_one_2006-6-13_14-42-6,/media/MAX/Data/kdiba/gor01/one/2006-6-13_14-42-6,SessionBatchProgress.COMPLETED,None,43,13,39,23,True,,,,False
6,kdiba,gor01,two,2006-6-07_16-40-19,kdiba_gor01_two_2006-6-07_16-40-19,/media/MAX/Data/kdiba/gor01/two/2006-6-07_16-4...,SessionBatchProgress.COMPLETED,None,42,145,40,239,True,,,,True
8,kdiba,gor01,two,2006-6-08_21-16-25,kdiba_gor01_two_2006-6-08_21-16-25,/media/MAX/Data/kdiba/gor01/two/2006-6-08_21-1...,SessionBatchProgress.COMPLETED,None,40,30,40,47,True,,,,False
9,kdiba,gor01,two,2006-6-09_22-24-40,kdiba_gor01_two_2006-6-09_22-24-40,/media/MAX/Data/kdiba/gor01/two/2006-6-09_22-2...,SessionBatchProgress.COMPLETED,None,51,78,43,299,True,,,,False
10,kdiba,gor01,two,2006-6-12_16-53-46,kdiba_gor01_two_2006-6-12_16-53-46,/media/MAX/Data/kdiba/gor01/two/2006-6-12_16-5...,SessionBatchProgress.COMPLETED,None,41,41,40,22,True,,,,False
11,kdiba,gor01,two,2006-6-13_15-22-3,kdiba_gor01_two_2006-6-13_15-22-3,/media/MAX/Data/kdiba/gor01/two/2006-6-13_15-22-3,SessionBatchProgress.COMPLETED,None,41,20,42,14,True,,,,False
12,kdiba,vvp01,one,2006-4-09_17-29-30,kdiba_vvp01_one_2006-4-09_17-29-30,/media/MAX/Data/kdiba/vvp01/one/2006-4-09_17-2...,SessionBatchProgress.COMPLETED,None,63,27,44,35,True,,,,False


In [ ]:
for index, row in good_only_batch_progress_df.iterrows():
    if row['is_ready']:
        row_context = row['context']
        print(f"context: {row_context}")
        row_context in user_is_replay_good_annotations
    

# Build `global_batch_run` pre-loading results (before execution)

In [5]:
# global_batch_result = loadData('global_batch_result.pkl')
global_batch_run = run_diba_batch(global_data_root_parent_path, execute_all=False, extant_batch_run=global_batch_run, debug_print=False)
# print(f'global_batch_result: {global_batch_run}')
global_batch_run

resusing extant_batch_run: BatchRun(global_data_root_parent_path=PosixPath('/media/MAX/Data'), session_batch_status={IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_3-23-37')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '

BatchRun(global_data_root_parent_path=PosixPath('/media/MAX/Data'), session_batch_status={IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-07_11-26-53')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_3-23-37')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-13_14-42-6')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>: <SessionBatchProgress.NOT_STARTED: 'NOT_STARTED'>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-08_15-46-47')>: <Ses

# Run Batch Executions/Computations

In [6]:
# Skip sessions that aren't already computed. Refuse to do any computations (fail if computations are not done already)
# Compute as needed, and if computations were done save the result (caching it for future loading)

# Force Recompute:
#   - Compute always (this mode would be used after a bug was fixed in the computations and we want to be sure the latest version is used to produce the result) regardless of whether extant results exist. Once computation is complete, overwrite extant files if they exist.
#   - Same as above but do not save at all (just use in-memory result for something)
#   - Same as first one but save the result with a special filename (a variant) as not to overwrite the original



In [9]:
## I got it doing the bare-minimum loading and computations, so it should be ready to update the laps and constraint the placefields to those. Then we should be able to set up the replays at the same time.
# finally, we then finish by computing.
# force_reload = True
force_reload = False
result_handler = BatchSessionCompletionHandler(force_reload_all=force_reload, should_perform_figure_generation_to_file=False, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False)

## Execute with the custom arguments.
active_computation_functions_name_includelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=force_reload, saving_mode=PipelineSavingScheme.SKIP_SAVING, skip_extended_batch_computations=True, post_run_callback_fn=result_handler.on_complete_success_execution_session,
                                                                                        **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

enable_calling_completion_handler_for_previously_completed: True
basedir: /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl... Issue with pickled WindowsPath on Linux for path /media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl, performing pathlib workaround...
done.
enable_calling_completion_handler_for_previously_completed: True
basedir: /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15
active_data_mode_name: kdiba
Loading loaded session pickle file results : /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl... done.
Loading pickled pipeline success: /media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl.
property already present in pickled version. No need to save.
build_lap_only_short_long_bin_aligned_computation_configs(...):
	len(active_session_computation_configs): 1
	len(fin

KeyError: 'pf1D_Decoder'

In [ ]:
# last completed:
r'W:\Data\KDIBA\pin01\one\11-03_12-3-25'

In [ ]:
# Save to file:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

In [ ]:
## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
inst_fr_output_filename = 'long_short_inst_firing_rate_result.pkl'
global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# Save the all sessions instantaneous firing rate dict to the path:
saveData(global_batch_result_inst_fr_file_path, result_handler.across_sessions_instantaneous_fr_dict)

In [ ]:
#TODO 2023-07-12 10:12: - [ ] New save way after we save out current result and reload
result_handler.save_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename='across_session_result_long_short_inst_firing_rate.pkl')

In [ ]:
num_sessions = len(result_handler.across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')


In [ ]:
import dill

dill.dump(result_handler, str(global_batch_result_inst_fr_file_path))

In [ ]:
accumulated_lists = []

for a_ctxt, a_inst_result in result_handler.across_sessions_instantaneous_fr_dict.items():
	print(f'{a_ctxt} - {a_inst_result.Fig2_Laps_FR[2]}') # .cell_agg_inst_fr_list	
	# [(v.cell_agg_inst_fr_list.mean(), v.cell_agg_inst_fr_list.std(), v.cell_agg_inst_fr_list) for v in (a_inst_result.LxC_ThetaDeltaMinus, a_inst_result.LxC_ThetaDeltaPlus, a_inst_result.SxC_ThetaDeltaMinus, a_inst_result.SxC_ThetaDeltaPlus)]
	accumulated_lists.append([v.cell_agg_inst_fr_list for v in (a_inst_result.LxC_ThetaDeltaMinus, a_inst_result.LxC_ThetaDeltaPlus, a_inst_result.SxC_ThetaDeltaMinus, a_inst_result.SxC_ThetaDeltaPlus)])
	
# a_inst_result.LxC_ReplayDeltaMinus, a_inst_result.LxC_ReplayDeltaPlus, a_inst_result.SxC_ReplayDeltaMinus, a_inst_result.SxC_ReplayDeltaPlus
# a_inst_result.LxC_ThetaDeltaMinus, a_inst_result.LxC_ThetaDeltaPlus, a_inst_result.SxC_ThetaDeltaMinus, a_inst_result.SxC_ThetaDeltaPlus
accumulated_lists


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

all_LxC_ThetaDeltaMinus = SpikeRateTrends([], []) # empty lists
all_LxC_ThetaDeltaMinus.inst_fr_df_list.append(

In [ ]:
np.vstack(accumulated_lists)

In [ ]:

v.cell_agg_inst_fr_list


In [ ]:
# Get output files:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
batch_progress_df.
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

## Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

# Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]



completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths